In [2]:
import pandas as pd

In [3]:
A = pd.read_csv('./X_larger_wavelength.csv')
# 选取特定行
# 选取第1行
A.iloc[0]
# 选取前两行数据
A.iloc[0:2]
# 选取前三列数据
A.iloc[0:10, 105:125]
AA = A.iloc[0:10, 105:125] # 训练集
AA
BB = A.iloc[10:14, 105:125] # 测试集
BB

,245,244,243,242,241,240,239,238,237,236,235,234,233,232,231,230,229,228,227,226
10,0.148,0.150,0.152,0.161,0.165,0.176,0.176,0.175,0.182,0.191,0.196,0.195,0.204,0.211,0.211,0.213,0.219,0.216,0.216,0.226
11,0.217,0.229,0.231,0.242,0.252,0.262,0.270,0.286,0.297,0.304,0.314,0.323,0.330,0.336,0.351,0.357,0.358,0.367,0.376,0.381
12,0.250,0.268,0.281,0.297,0.307,0.330,0.334,0.354,0.366,0.385,0.388,0.403,0.414,0.427,0.449,0.452,0.453,0.467,0.479,0.493
13,0.280,0.303,0.320,0.337,0.366,0.392,0.408,0.429,0.446,0.469,0.486,0.511,0.530,0.548,0.559,0.570,0.589,0.600,0.615,0.631


In [4]:
# https://blog.csdn.net/li_huifei/article/details/78467689
# 汽油的NIR吸收光谱与其RON值存在回归关系，通过偏最小二乘法可以建立回归方程，
# 通过分析NIR光谱较为准确的定量分析出RON含量。
# 虽然在样本数量较少的情况下误差会稍稍有点大，但是由偏最小二乘法同时关注了输
# 入量自身的信息和输入与输出的关系，较大程度上尽可能多的利用了样本中有限的信息，同
# 时体现出了偏最小二乘相对于普通PCA的优越性。普通PCA只关注输入量，旨在从输入中
# 用尽量少维度的向量尽可能提取出输入量中的所有信息，在这个过程中忽略了输入与输出的
# 相关关系，而PLS在对输入进行降维的同时要求输入与输出量的相关系数尽量大，在样本
# 数量不足时尽可能多的保证了回归分析的准确度。
# 自变量是光谱，因变量是RON含量
import csv
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
# from sklearn.decomposition import RandomizedPCA
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.cross_decomposition import PLSRegression
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

# A = np.loadtxt('./PLS/X.csv', delimiter=',')#loadtxt读入数据时，把表头去掉
A = pd.read_csv('./X_larger_wavelength.csv')
# 选取特定行
# 选取第1行
A.iloc[0]
# 选取前两行数据
A.iloc[0:2]
# 选取前三列数据
A.iloc[0:10, 105:125]
AA = A.iloc[0:10, 105:125] # 训练集
AA
BB = A.iloc[10:14, 105:125] # 测试集
BB

A = pd.read_csv('./X1.csv')
print(A.shape)
# Olefin= np.loadtxt('Olefin.csv',delimiter=',')
# print (Olefin.shape)
# Y = np.loadtxt('./PLS/Y.csv',delimiter=',')
# print (RON.shape)
Y = pd.read_csv('./Y1.csv')


# OR = np.array((RON,Olefin)).T
# print (OR.shape)
# nm = np.loadtxt('nm.csv',delimiter=',')
def error(y_predict, y_test):  # 定义计算误差平方和函数
    errs = []
    # print("y_predict.size: ", len(y_predict))
    for i in range(len(y_predict)):
        # print("y_predict0:", y_predict0)
        # print("y_predict[0]:", y_predict[0])
        # print("y_test[0]:", y_test.iloc[0])
        e = (y_predict[i] - y_test.iloc[i]) ** 2
        errs.append(e)
    return sum(errs)


x_train, x_test, y_train, y_test = train_test_split(
    AA, Y, test_size=0.3)  # 划分训练集测试集
x_train_st = preprocessing.scale(x_train)  # 数据标准化
n_components = 0
shape = x_train_st.shape[1]

'''
y_predict0: [[45.42294446]
 [87.79694196]
 [ 5.15276626]
 [81.26289003]
 [55.54740263]
 [67.15619902]
 [17.66085564]]
y_train:    nongdu
3      40
9     100
0      10
7      80
4      50
5      60
1      20
'''
n_components_arr = []
dict_components = {}
while n_components < x_train_st.shape[1]:
    n_components += 1
    # n_components_arr.append(n_components)
    pls2 = PLSRegression(n_components=n_components)  # 计算SS
    # print("y_train:", y_train)
    # print("y_train[0]:", y_train.iloc[0] ** 2)

    # pls2.fit(x_train_st, y_train)
    pls2.fit(x_train, y_train)
    # y_predict0 = pls2.predict(x_train_st)
    y_predict0 = pls2.predict(x_train)
    # print("predict0:", y_predict0)
    # print("y_train:", y_train)

    '''
    predict0: [[66.50447647]
     [80.60612714]
     [87.13815318]
     [74.53804362]
     [31.78769359]
     [54.90007442]
     [ 4.52543158]]
    y_train:    nongdu
    5      60
    7      80
    9     100
    6      70
    2      30
    4      50
    0      10
    '''
    SS = error(y_predict0, y_train)
    y_predict1 = []
    for i in range(x_train_st.shape[0]):  # 计算PRESS

        n_components1 = n_components + 1
        # x_train_st1 = np.delete(x_train_st, i, 0)
        x_train_st1 = np.delete(x_train, i, 0)
        # y_train_st1 = np.delete(y_train_st,i,0)
        pls2 = PLSRegression(n_components=n_components1)
        # pls2.fit(x_train_st, y_train)
        pls2.fit(x_train, y_train)
        # y_predict11 = pls2.predict(x_train_st[i].reshape(1, -1))
        y_predict11 = pls2.predict(x_train[i].reshape(1, -1))
        y_predict1.append(y_predict11)
    # 结果表明回归方程在训练集上表现良好，误差平方和非常小，拟合效果很好。在计算PRESS
    # 和ss时所得到的预测值和真实值描点汇成如下两图:
    # print("predict1:", y_predict1)
    # print("y_train:", y_train)
    '''
    predict1: [array([[60.58776224]]), array([[82.42153724]]), array([[98.14180922]]), array([[70.90227744]]), array([[29.32838734]]), array([[47.77641892]]), array([[10.84180759]])]
    y_train:    nongdu
    5      60
    7      80
    9     100
    6      70
    2      30
    4      50
    0      10
    '''
    # 以下3行代码用于数据预处理
    y_predict1_1 = []
    for i in range(len(y_predict1)):
        y_predict1_1.append(y_predict1[i][0])
    PRESS = error(y_predict1_1, y_train)
    dict_components[n_components] = PRESS
    Qh = 1 - float(PRESS / SS)
    if Qh < 0.0985:  # 判断精度
        plt.figure(1)
        plt.scatter(y_predict0, y_train)
        plt.figure(2)
        plt.scatter(y_predict1, y_train)
        print('the Qh is ', Qh)
        print('the PRESS is', PRESS)
        print('the SS is', SS)
        break

print('n_components is ', n_components + 1)
print("dict_components:", dict_components)


(10, 12)


ValueError: Shape of passed values is (6, 20), indices imply (7, 20)

In [5]:
# 对已知的AA数据中的测试集进行验证
# x_train_st
# x_test
# x_test_predict = pls2.predict(x_test)
# x_test_predict
x_train, x_test, y_train, y_test = train_test_split(
    AA, Y, test_size=0.3)  # 划分训练集测试集
# x_train_stt = preprocessing.scale(x_train)  # 数据标准化
# pls3 = PLSRegression(n_components=9)
# pls3.fit(x_train_stt, y_train)
x_test_stt = preprocessing.scale(x_test)
y_predict_tt = pls2.predict(x_test_stt)
y_test, y_predict_tt


(   nongdu
 7      80
 2      30
 9     100,
 array([[  20.51492861],
        [-113.08447619],
        [  33.65455252]]))

In [6]:
# 对未知浓度的BB浓度进行预测
BB
# x_train, x_test, y_train, y_test = train_test_split(
#     AA, Y, test_size=0.3)  # 划分训练集测试集
# x_train_stt = preprocessing.scale(x_train)  # 数据标准化
# pls3 = PLSRegression(n_components=9)
# pls3.fit(x_train_stt, y_train)
x_test_stt = preprocessing.scale(BB)
y_predict_tt = pls2.predict(BB)
y_predict_tt

array([[-7.0412803 ],
       [ 0.63923558],
       [ 5.69769794],
       [11.70148259]])

In [7]:
test = pd.read_csv('./test.csv')
test_std = preprocessing.scale(test)  # 数据标准化
test_std
predict_test = pls2.predict(test_std)
predict_test

ValueError: operands could not be broadcast together with shapes (5,12) (20,) (5,12) 